In [26]:
import cv2
import glob
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm 
import os 
import pickle
from PIL import Image
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import tensorflow as tf

In [1]:
path = './'

In [27]:
from tensorflow import keras
from tensorflow.keras import backend as K

In [28]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
image_path = os.path.join(path, 'VOC_PCB/JPEGImages')
print(image_path)
annotate_path = os.path.join(path, 'VOC_PCB/Annotations')
print(annotate_path)
print(annotate_path)

/kaggle/input/pcb-defect-dataset/VOC_PCB/JPEGImages
/kaggle/input/pcb-defect-dataset/VOC_PCB/Annotations
/kaggle/input/pcb-defect-dataset/VOC_PCB/Annotations


In [1]:
from utils import *

  0%|          | 0/10668 [00:00<?, ?it/s]

  0%|          | 0/10668 [00:00<?, ?it/s]

In [55]:
bbox_for_all_image = bbox_for_all_image[:5334]
boxarr=boxarr[:5334]

In [56]:
dataset = tf.data.Dataset.from_tensor_slices((bbox_for_all_image,boxarr))

In [57]:
n=1500
dataset = dataset.shuffle(n)
train_dataset = dataset.take(int(n*0.9)).batch(100)
test_dataset = dataset.skip(int(n*0.9)).batch(100)

In [65]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, BatchNormalization, Activation, Dense, Dropout
from tensorflow.keras.layers import Conv2D, Conv2DTranspose
from tensorflow.keras.layers import MaxPooling2D, GlobalMaxPool2D
from tensorflow.keras.layers import concatenate, add
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

In [70]:
def conv2d_block(input_tensor, n_filters, kernel_size = 3, batchnorm = True):
    """Function to add 2 convolutional layers with the parameters passed to it"""
    # first layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # second layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
    
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)

    return x

def get_unet(input_img, n_filters = 16, dropout = 0.1, batchnorm = True):

    c1 = conv2d_block(input_img, n_filters * 1, kernel_size = 3, batchnorm = batchnorm)
    p1 = MaxPooling2D((2, 2))(c1)
    p1 = Dropout(dropout)(p1)

    c2 = conv2d_block(p1, n_filters * 2, kernel_size = 3, batchnorm = batchnorm)
    p2 = MaxPooling2D((2, 2))(c2)
    p2 = Dropout(dropout)(p2)

    c3 = conv2d_block(p2, n_filters * 4, kernel_size = 3, batchnorm = batchnorm)
    p3 = MaxPooling2D((2, 2))(c3)
    p3 = Dropout(dropout)(p3)

    c4 = conv2d_block(p3, n_filters * 8, kernel_size = 3, batchnorm = batchnorm)
    p4 = MaxPooling2D((2, 2))(c4)
    p4 = Dropout(dropout)(p4)

    c5 = conv2d_block(p4, n_filters = n_filters * 16, kernel_size = 3, batchnorm = batchnorm)

    u6 = Conv2DTranspose(n_filters * 8, (3, 3), strides = (2, 2), padding = 'same')(c5)
    u6 = concatenate([u6, c4])
    u6 = Dropout(dropout)(u6)
    c6 = conv2d_block(u6, n_filters * 8, kernel_size = 3, batchnorm = batchnorm)

    u7 = Conv2DTranspose(n_filters * 4, (3, 3), strides = (2, 2), padding = 'same')(c6)
    u7 = concatenate([u7, c3])
    u7 = Dropout(dropout)(u7)
    c7 = conv2d_block(u7, n_filters * 4, kernel_size = 3, batchnorm = batchnorm)

    u8 = Conv2DTranspose(n_filters * 2, (3, 3), strides = (2, 2), padding = 'same')(c7)
    u8 = concatenate([u8, c2])
    u8 = Dropout(dropout)(u8)
    c8 = conv2d_block(u8, n_filters * 2, kernel_size = 3, batchnorm = batchnorm)

    u9 = Conv2DTranspose(n_filters * 1, (3, 3), strides = (2, 2), padding = 'same')(c8)
    u9 = concatenate([u9, c1])
    u9 = Dropout(dropout)(u9)
    c9 = conv2d_block(u9, n_filters * 1, kernel_size = 3, batchnorm = batchnorm)

    outputs = Conv2D(7, (1, 1), activation='sigmoid')(c9)
    model = Model(inputs=[input_img], outputs=[outputs])
    return model

input_img = Input((128, 128, 1), name='img')
model = get_unet(input_img, n_filters=16, dropout=0.05, batchnorm=True)
model.compile(optimizer=Adam(), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=["accuracy"])

2021-10-19 09:39:47.553307: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:301] Initialize GrpcChannelCache for job worker -> {0 -> 10.0.0.2:8470}
2021-10-19 09:39:47.553384: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:301] Initialize GrpcChannelCache for job localhost -> {0 -> localhost:30246}
2021-10-19 09:39:47.555766: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:301] Initialize GrpcChannelCache for job worker -> {0 -> 10.0.0.2:8470}
2021-10-19 09:39:47.555810: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:301] Initialize GrpcChannelCache for job localhost -> {0 -> localhost:30246}


In [71]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
img (InputLayer)                [(None, 128, 128, 1) 0                                            
__________________________________________________________________________________________________
conv2d_20 (Conv2D)              (None, 128, 128, 16) 160         img[0][0]                        
__________________________________________________________________________________________________
batch_normalization_19 (BatchNo (None, 128, 128, 16) 64          conv2d_20[0][0]                  
__________________________________________________________________________________________________
activation_19 (Activation)      (None, 128, 128, 16) 0           batch_normalization_19[0][0]     
____________________________________________________________________________________________

In [73]:
callbacks = [EarlyStopping(patience=10, verbose=1),ReduceLROnPlateau(factor=0.001, patience=10, min_lr=0.0001, verbose=1),
             ModelCheckpoint('model best.h5', verbose=1, save_best_only=True, save_weights_only=False,
                             monitor='val_loss',save_freq='epoch',mode='loss')]

history = model.fit(train_dataset,epochs=2000,validation_data=test_dataset,callbacks=[callback],batch_size=32,)

Epoch 1/2000
14/14 [==============================] - 2s 182ms/step - loss: 0.1793 - accuracy: 0.9943 - val_loss: 0.1161 - val_accuracy: 0.9941

Epoch 00001: val_loss did not improve from 0.10080
Epoch 2/2000
14/14 [==============================] - 2s 178ms/step - loss: 0.1594 - accuracy: 0.9943 - val_loss: 0.1072 - val_accuracy: 0.9941

Epoch 00002: val_loss did not improve from 0.10080
Epoch 3/2000
14/14 [==============================] - 3s 193ms/step - loss: 0.1428 - accuracy: 0.9943 - val_loss: 0.0991 - val_accuracy: 0.9941

Epoch 00003: val_loss improved from 0.10080 to 0.09910, saving model to model1500epoch.h5
Epoch 4/2000
14/14 [==============================] - 3s 198ms/step - loss: 0.1281 - accuracy: 0.9946 - val_loss: 0.0939 - val_accuracy: 0.9941

Epoch 00004: val_loss improved from 0.09910 to 0.09389, saving model to model1500epoch.h5
Epoch 5/2000
14/14 [==============================] - 3s 219ms/step - loss: 0.1171 - accuracy: 0.9945 - val_loss: 0.0881 - val_accuracy: 0